In [4]:
%run "libraries.ipynb"
import networkx as nx
from IPython.html import widgets
from IPython.display import display, HTML

# importing datasets

In [17]:
# list of page names
pages = codecs.open("data/pagenames.txt","r", "utf-8-sig").readlines()
pages = map(lambda x: x.strip(), pages)

# page graph obtained by projecting page-editor bi-partite graph
pages_graph = nx.read_gexf("data/pages-linked-by-coeditors.gexf")

# page graph obtained by projecting page-editor bi-partite graph
pages_editors_graph = nx.read_gexf("data/pages-editors.gexf")

In [29]:
def table_to_html(data, cols=[]):
    html = "<table>"
    html += "<tr>"
    for column_content in cols:
        html += "<th>%s</th>" % (column_content)
    html += "</tr>"
    
    for d in data:
        html += "<tr>"
        for column_content in d:
            html += "<td>%s</td>" % (column_content)
        html += "</tr>"
    html += "</table>"
        
    return HTML(html) 

def show_info(page):
    display(HTML("<h2>%s</h2>" % (page)))
    
    #display(HTML("<div style=\"float:left\">"))    
    display(HTML("<h3>co-edited pages</h3>"))

    nb = sorted(pages_graph["p:%s" % (page)].items(),
            key=lambda (k,x): -int(x["coeditors"]))

    data = []

    # calculate rank in neighbor top co-edited ranking
    for name, info in nb:
        nb_mirror = sorted(pages_graph[name].items(),
                key=lambda (k,x): -int(x["coeditors"]))
        nb_mirror = [ x[0] for x in nb_mirror ]
        
        editors = pages_editors_graph[name]        
        info["editors"] = len(editors)
        info["exclusive editors"] = len([n for n in editors if len(pages_editors_graph[n]) == 1 ])

        info["ranking"] = nb_mirror.index("p:%s" % (page)) + 1

    #print nb

    for name, info in nb[0:10]:
        data.append([ "<a href=\"http://en.wikipedia.org/wiki/{0}\" target=\"_blank\">{0}</a>".format(name.split(":")[1]),
                     info["editors"],
                     info["coeditors"],
                     float(info["coeditors"]) / float(info["editors"]),
                     info["exclusive editors"],
                     info["ranking"]])

    display(table_to_html(data, ["page name", "editors", "co-editors", "co-editors/editors", "exclusive editors" ,"ranking"]))
    #display(HTML("</div>"))    

    #display(HTML("<div style=\"float:left\">"))    
    display(HTML("<h3>ranked first in</h3>"))

    nb_list = [ x[0] for x in nb ]
    data = []

    nb2 = sorted(nb, key=lambda (x): x[1]["ranking"])
    for name, info in nb2[0:10]:
        editors = pages_editors_graph[name]        
        info["editors"] = len(editors)
        info["exclusive editors"] = len([n for n in editors if len(pages_editors_graph[n]) == 1 ])

        data.append([ "<a href=\"http://en.wikipedia.org/wiki/{0}\" target=\"_blank\">{0}</a>".format(name.split(":")[1]),
                     info["editors"],
                     info["coeditors"],
                     float(info["coeditors"]) / float(info["editors"]),
                     info["exclusive editors"],
                     info["ranking"]])  

    display(table_to_html(data, ["page name", "editors", "co-editors", "co-editors/editors", "exclusive editors" ,"ranking"]))
#     display(HTML("</div>"))    
#     display(HTML("<div style=\"clear:both\"></div>"))
    
    display(HTML("<h3>pageviews and revisions</h3>"))
    display(HTML("<h3>top editors</h3>"))   
#show_info("Pi")

# explorer

In [30]:
w = widgets.SelectWidget(description="page", options=pages)
widgets.interact(show_info, page=w);

page name,editors,co-editors,co-editors/editors,exclusive editors,ranking
Triangle,916,144,0.157205240175,295,8
Astronomy,1239,137,0.110573042776,583,8
Pi,1799,120,0.0667037242913,886,11
Mathematics,1509,117,0.0775347912525,676,10
Trigonometry,854,108,0.126463700234,323,10
Geometry,599,107,0.178631051753,180,9
Pythagorean theorem,947,104,0.109820485744,330,12
Circle,807,98,0.121437422553,253,14
Fractal,760,92,0.121052631579,321,9
Angle,469,80,0.170575692964,132,13


page name,editors,co-editors,co-editors/editors,exclusive editors,ranking
Crystal system,101,26,0.257425742574,27,5
2D computer graphics,139,27,0.194244604317,36,6
Straightedge,59,13,0.220338983051,15,6
Quasicrystal,200,34,0.17,74,7
Triangle,916,144,0.157205240175,295,8
Astronomy,1239,137,0.110573042776,583,8
Geometry,599,107,0.178631051753,180,9
Fractal,760,92,0.121052631579,321,9
Tessellation,416,73,0.175480769231,111,9
Pattern,252,47,0.186507936508,92,9
